In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os
from tqdm import tqdm # For a nice progress bar

# --- 1. CONFIGURATION ---
class Config:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    num_workers = 2
    dataset_path = '/kaggle/input/dgm-animals/Animals_data/animals/animals'
    
    # Model and Training Parameters
    num_classes = 90
    batch_size = 32 # Smaller batch size for a large model like ResNet-50
    num_epochs = 15
    lr = 0.001
    
    # Data Split
    val_split = 0.2
    
    # Output
    output_dir = '/kaggle/working/Q7_ResNet_Classifier'

config = Config()
os.makedirs(config.output_dir, exist_ok=True)
print(f"Configuration loaded. Using device: {config.device}")

Configuration loaded. Using device: cuda


In [2]:
# --- DATA TRANSFORMS ---
# For training, we use data augmentation. For validation, we only resize and normalize.
# The normalization values are standard for models pre-trained on ImageNet.
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- CREATE AND SPLIT DATASET ---
# We create two instances of the dataset with different transforms
full_dataset_train = datasets.ImageFolder(root=config.dataset_path, transform=data_transforms['train'])
full_dataset_val = datasets.ImageFolder(root=config.dataset_path, transform=data_transforms['val'])

# Split the dataset indices
dataset_size = len(full_dataset_train)
val_size = int(config.val_split * dataset_size)
train_size = dataset_size - val_size
train_indices, val_indices = random_split(range(dataset_size), [train_size, val_size])

# Create subsets for train and validation
train_subset = torch.utils.data.Subset(full_dataset_train, train_indices)
val_subset = torch.utils.data.Subset(full_dataset_val, val_indices)

# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
val_loader = DataLoader(val_subset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

print(f"Data prepared. Training samples: {len(train_subset)}, Validation samples: {len(val_subset)}")

Data prepared. Training samples: 4301, Validation samples: 1075


In [3]:
# --- LOAD PRE-TRAINED RESNET-50 ---
# Load ResNet-50 with the latest recommended weights
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# --- FREEZE PRE-TRAINED LAYERS (Optional, but good practice for fine-tuning) ---
# for param in model.parameters():
#     param.requires_grad = False

# --- REPLACE THE FINAL CLASSIFICATION LAYER ---
# Get the number of input features for the classifier
num_ftrs = model.fc.in_features

# Create a new final layer for our 90 classes
model.fc = nn.Linear(num_ftrs, config.num_classes)

# Move the model to the configured device
model = model.to(config.device)

print("ResNet-50 model loaded and final layer replaced for fine-tuning.")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 230MB/s]


ResNet-50 model loaded and final layer replaced for fine-tuning.


In [4]:
# --- LOSS FUNCTION AND OPTIMIZER ---
criterion = nn.CrossEntropyLoss()
# We will only train the parameters of the final layer, but for full fine-tuning,
# you would pass model.parameters() to the optimizer.
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# --- TRAINING AND VALIDATION LOOP ---
best_val_accuracy = 0.0
print("\nStarting training...")

for epoch in range(config.num_epochs):
    # --- Training Phase ---
    model.train()
    running_loss = 0.0
    
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.num_epochs} [T]")
    for inputs, labels in train_pbar:
        inputs, labels = inputs.to(config.device), labels.to(config.device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_train_loss = running_loss / len(train_loader.dataset.indices)

    # --- Validation Phase ---
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.num_epochs} [V]")
    with torch.no_grad():
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(config.device), labels.to(config.device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            
            running_loss += loss.item() * inputs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_val_loss = running_loss / len(val_loader.dataset.indices)
    epoch_val_accuracy = accuracy_score(all_labels, all_preds)

    print(f"Epoch {epoch+1}/{config.num_epochs} -> "
          f"Train Loss: {epoch_train_loss:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f} | "
          f"Val Accuracy: {epoch_val_accuracy:.4f}")
    
    # Save the model if it has the best validation accuracy so far
    if epoch_val_accuracy > best_val_accuracy:
        best_val_accuracy = epoch_val_accuracy
        torch.save(model.state_dict(), os.path.join(config.output_dir, 'best_model.pth'))
        print(f"*** New best model saved with accuracy: {best_val_accuracy:.4f} ***")

print("\nTraining finished.")

# --- FINAL METRIC CALCULATION ---
# Load the best performing model
model.load_state_dict(torch.load(os.path.join(config.output_dir, 'best_model.pth')))
model.eval()

# We already have the predictions and labels from the last validation epoch of the best model
# (or we could re-calculate on the val set if needed)
final_accuracy = accuracy_score(all_labels, all_preds)
# For F1-score, 'weighted' accounts for label imbalance by calculating metrics for each
# label, and finding their average weighted by support (the number of true instances for each label).
final_f1_score = f1_score(all_labels, all_preds, average='weighted')

print("\n" + "="*40)
print("      Final Model Performance")
print("="*40)
print(f"Classification Accuracy: {final_accuracy:.4f}")
print(f"Weighted F1 Score:       {final_f1_score:.4f}")
print("="*40)


Starting training...


Epoch 1/15 [V]: 100%|██████████| 34/34 [00:12<00:00,  2.82it/s]


Epoch 1/15 -> Train Loss: 3.1436 | Val Loss: 2.6255 | Val Accuracy: 0.4047
*** New best model saved with accuracy: 0.4047 ***


Epoch 2/15 [V]: 100%|██████████| 34/34 [00:08<00:00,  3.99it/s]


Epoch 2/15 -> Train Loss: 2.0837 | Val Loss: 2.3495 | Val Accuracy: 0.4335
*** New best model saved with accuracy: 0.4335 ***


...

Epoch 13/15 -> Train Loss: 0.6864 | Val Loss: 0.9275 | Val Accuracy: 0.7563


Epoch 14/15 [V]: 100%|██████████| 34/34 [00:08<00:00,  4.18it/s]


Epoch 14/15 -> Train Loss: 0.6355 | Val Loss: 1.1445 | Val Accuracy: 0.7572


Epoch 15/15 [V]: 100%|██████████| 34/34 [00:07<00:00,  4.28it/s]


Epoch 15/15 -> Train Loss: 0.6489 | Val Loss: 1.2863 | Val Accuracy: 0.6949

Training finished.

      Final Model Performance
Classification Accuracy: 0.6949
Weighted F1 Score:       0.6941
